In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import os
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
 
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("./egyesitett_jo.csv")
data["Date"] = pd.to_datetime(data["Date"])
data

,Date,Close,Volume,Open,High,Low,Name
0,2023-10-19,128.40,60961360,130.57,132.24,127.47,AMZN
1,2023-10-18,128.13,42699480,129.90,130.67,127.51,AMZN
2,2023-10-17,131.47,49344550,130.39,132.58,128.71,AMZN
3,2023-10-16,132.55,42832920,130.69,133.07,130.43,AMZN
4,2023-10-13,129.79,45824690,132.98,133.31,128.95,AMZN
...,...,...,...,...,...,...,...
12580,2013-10-25,11.31,113661738,11.61,11.63,11.12,TSLA
12581,2013-10-24,11.54,161740419,11.00,11.63,10.86,TSLA
12582,2013-10-23,10.97,199788950,11.26,11.45,10.68,TSLA
12583,2013-10-22,11.44,170788865,11.37,11.85,11.07,TSLA


In [3]:
amazon_close = np.array(data[data["Name"] == "AMZN"].sort_values(by = "Date")[["Close"]].values)
apple_close = np.array(data[data["Name"] == "AAPL"].sort_values(by = "Date")[["Close"]].values)
microsoft_close = np.array(data[data["Name"] == "MSFT"].sort_values(by = "Date")[["Close"]].values)
starbucks_close = np.array(data[data["Name"] == "SBUX"].sort_values(by = "Date")[["Close"]].values)
tesla_close = np.array(data[data["Name"] == "TSLA"].sort_values(by = "Date")[["Close"]].values)

# AIC és BIC kipróbálása a Tesla részvényére

In [4]:
train_size = int(len(tesla_close) * 0.9)
train, test = tesla_close[0:train_size], tesla_close[train_size:]

In [5]:
order_aic_bic=[]

for d in [0, 1, 2]:
    for p  in range(6):
        for q in range(6):
            try:
                model=ARIMA(train, order=(p,d,q))
                results=model.fit()

                order_aic_bic.append((p,d,q,results.aic,results.bic))
            except:
                order_aic_bic.append((p,d,q,None,None))
            
order_df=pd.DataFrame(order_aic_bic,columns=['p','d','q','AIC','BIC']).sort_values('AIC')
order_df

,p,d,q,AIC,BIC
71,5,1,5,13707.196753,13770.170526
101,4,2,5,13711.094658,13768.339124
107,5,2,5,13714.504627,13777.473540
65,4,1,5,13714.714055,13771.962940
106,5,2,4,13720.816142,13778.060608
...,...,...,...,...,...
5,0,0,5,19290.447070,19330.524381
3,0,0,3,20200.890116,20229.516766
2,0,0,2,22093.096726,22115.998047
1,0,0,1,24493.625350,24510.801340


In [6]:
best_p, best_d, best_q = order_df.iloc[0][["p", "d", "q"]]
best_p, best_d, best_q

(5.0, 1.0, 5.0)

In [7]:
#MSE kiértékelése a legjobb konfigurációval
model = ARIMA(train, order=(best_p, best_d, best_q))
model_fit = model.fit()

predictions = []
for t in range(0, len(test)):
    yhat = model_fit.forecast()
    predictions.append(yhat)


    model_fit = model_fit.append(test[t:t+1], refit=False)
    
mean_squared_error(predictions, test)

56.43325226918525

In [8]:
model = ARIMA(train, order=(4, 1, 3))
model_fit = model.fit()

predictions = []
for t in range(0, len(test)):
    yhat = model_fit.forecast()
    predictions.append(yhat)


    model_fit = model_fit.append(test[t:t+1], refit=False)
    
mean_squared_error(predictions, test)

53.49616415351169